# Testing a general data reader for AQUA 
## (working on Levante for now)

In [1]:
import intake
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from aqua import Reader
from aqua.reader import catalogue

First let's see what is available (this function also returns the catalogue, that's why we use a semicolon)

In [2]:
catalogue();

ICON	R02B09	5km coupled run, main ICON nextGEMS C2 run
ICON	hpx	Some Healpix data
IFS	tco2559-ng5	4km experiment, coupled with FESOM


Now let's load some IFS data

In [8]:
reader = Reader(model="IFS", exp="tco2559-ng5")
data = reader.retrieve()

These are raw IFS data on the original grid

In [9]:
data

<xarray.Dataset>
Dimensions:  (time: 8329, value: 26306560, level: 5)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2021-01-01
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

Instantiate a reader for ICON data specifying that we will want to interpolate to a 0.1 deg grid (the weights have been prepared as "010")

In [4]:
reader = Reader(model="ICON", exp="R02B09", regrid="010")

Load the actual data. By default these data have not been regridded yet. 

You could ask to regrid them directly by specifying the argument `regrid=True` but please be warned that without a selection on dates this will take longer. It may be more efficient to first load the data, select and then regrid.

In [5]:
data = reader.retrieve()

By default these data have not been regridded yet

In [6]:
data

<xarray.Dataset>
Dimensions:   (time: 37009, cell: 20971520)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20 ... 2022-03-01
Dimensions without coordinates: cell
Data variables: (12/34)
    cfh_lnd   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    clivi     (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    cllvi     (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    cptgzvi   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    hfls      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    hfss      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    ...        ...
    tas       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tauu      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tauv      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    ts        (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    uas       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    vas       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 152214
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

Now we actually regrid part of the data (the first 16 frames)

In [7]:
prr = reader.regrid(data.pr[0:16]) 

/home/b/b382075/mambaforge/envs/aqua/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
prr.load()

<xarray.DataArray 'pr' (time: 16, lat: 1800, lon: 3600)>
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.00163916e-06, 1.00176386e-06, 1.00188729e-06, ...,
         1.00125738e-06, 1.00138594e-06, 1.00151319e-06],
        [1.11976382e-06, 1.12023088e-06, 1.12069164e-06, ...,
         1.11832435e-06, 1.11881061e-06, 1.11929041e-06],
        [1.38140371e-06, 1.38206048e-06, 1.38270680e-06, ...,
         1.37937010e-06, 1.38005860e-06, 1.38073644e-06],
...
        [2.20724338e-06, 2.20326709e-06, 2.19927966e-06, ...,
         2.21910941e-06, 2.21516414e-06, 2.21120893e-06],
        [1.60897160e-06, 1.60531638e-06, 1.60165073e-06, ...,
         1.61987827e-06, 1.61625217e-06, 1.61261674e-06],
        [1.25914593e-06, 1.25801852e-06, 1.25688825e-06, ...,
         1.26251212e-06, 1.26139262e-06, 1.26027059e-06]],

       [[8.13681222e-08, 8.15018432e-08, 8.16331834e-08, ...,
         8.09525319e-08, 8.10934812e-08, 8.12320063e-08],
        [2.82318386e-07, 2.83194854e-07, 2.84058964e-07, ...,
         2.79613891e-07, 2.80528001e-07, 2.81429466e-07],
        [7.03483886e-07, 7.04205031e-07, 7.04921347e-07, ...,
         7.01290734e-07, 7.02026814e-07, 7.02757839e-07],
        ...,
        [1.31891475e-06, 1.31853008e-06, 1.31814535e-06, ...,
         1.32006878e-06, 1.31968406e-06, 1.31929939e-06],
        [1.25115769e-06, 1.25059952e-06, 1.25003836e-06, ...,
         1.25281473e-06, 1.25226524e-06, 1.25171291e-06],
        [1.04606751e-06, 1.04537932e-06, 1.04468742e-06, ...,
         1.04811048e-06, 1.04743302e-06, 1.04675206e-06]]])
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-20T07:30:00
  * lat      (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.5 359.6 359.7 359.8 359.9

In [9]:
prr.to_netcdf("test.nc")

In [7]:
# cdo genycon,data/r180x91/climate_CRU_t2m.nc ../../destine/nextgems/ngc2009_atm_2d_30min_inst_20220225T000000Z.tas.nc weights_icon_ycon.nc

Finally, a test with healpix data

In [4]:
reader = Reader(model="ICON", exp="hpx")
data = reader.retrieve()

In [12]:
data

<xarray.Dataset>
Dimensions:             (time: 8640, pix: 3145728, level: 90)
Coordinates:
  * level               (level) float64 nan 1.0 2.0 3.0 ... 86.0 87.0 88.0 89.0
  * time                (time) datetime64[ns] 2020-07-23T00:30:00 ... 2021-01-19
Dimensions without coordinates: pix
Data variables:
    10m_wind_speed      (time, pix) float32 dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    cli                 (time, level, pix) float32 dask.array<chunksize=(1, 10, 262144), meta=np.ndarray>
    clivi               (time, pix) float32 dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    cllvi               (time, pix) float32 dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    clw                 (time, level, pix) float32 dask.array<chunksize=(1, 10, 262144), meta=np.ndarray>
    sea_level_pressure  (time, pix) float32 dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    tas                 (time, pix) float32 dask.array<chunksize=(1, 1048576), meta=np.ndarray>